In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d crawford/emnist -f emnist-balanced-train.csv
!kaggle datasets download -d crawford/emnist -f emnist-balanced-test.csv

 89% 33.0M/37.2M [00:02<00:00, 19.1MB/s]
100% 37.2M/37.2M [00:02<00:00, 17.3MB/s]
 80% 5.00M/6.23M [00:00<00:00, 9.94MB/s]
100% 6.23M/6.23M [00:00<00:00, 12.3MB/s]


In [ ]:
! unzip emnist-balanced-train.csv.zip
! unzip emnist-balanced-test.csv.zip

Archive:  emnist-balanced-train.csv.zip
  inflating: emnist-balanced-train.csv  
Archive:  emnist-balanced-test.csv.zip
  inflating: emnist-balanced-test.csv  


In [ ]:
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('emnist-balanced-train.csv')
test = pd.read_csv('emnist-balanced-test.csv')
test.rename(columns={'41':'45'},inplace=True)

In [ ]:
col2=['45','0']
for i in range(1,784):
    col2.append('0.'+str(i))
train.columns=col2
test.columns=col2

In [ ]:
test.keys()

Index(['45', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8',
       ...
       '0.774', '0.775', '0.776', '0.777', '0.778', '0.779', '0.780', '0.781',
       '0.782', '0.783'],
      dtype='object', length=785)

In [ ]:
train.head()
# train.shape

,45,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.774,0.775,0.776,0.777,0.778,0.779,0.780,0.781,0.782,0.783
0,36,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,43,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
map=[10,11,38,39,41,16,35,32,17,18,19,27,46,20,22,28,23,33,37]
len(map)
trainlet=train.loc[train['45'].isin(map)]
testlet=test.loc[test['45'].isin(map)]

In [ ]:
trainlet.values[:, 1:] = np.abs(trainlet.values[:, 1:] - 255)
x=trainlet.values[1, 1:]
cv2_imshow(x.reshape(28,28))

In [ ]:
testlet.values[:, 1:] = np.abs(testlet.values[:, 1:] - 255)
x=testlet.values[1, 1:]
cv2_imshow(x.reshape(28,28))

In [ ]:
emo= pd.read_csv('finalemojis.csv')
print(emo.shape)
emo.rename(columns={'Target':'45'},inplace=True)
print(emo.keys())
trainemo, testemo = train_test_split(emo, test_size=0.2)
trainemo.reset_index(drop=True,inplace=True)
testemo.reset_index(drop=True,inplace=True)
train=pd.concat([trainlet,trainemo],axis=0,ignore_index=True)
test=pd.concat([testlet,testemo],axis=0,ignore_index=True)

(13119, 785)
Index(['45', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8',
       ...
       '0.774', '0.775', '0.776', '0.777', '0.778', '0.779', '0.780', '0.781',
       '0.782', '0.783'],
      dtype='object', length=785)


In [ ]:
len(train['45'].unique())
train.shape

(56095, 785)

In [ ]:
for i in range(0,15):
  df=train.loc[[i]]
  df.drop(['45'],axis=1,inplace=True)
  # df
  # df.keys()
  df=df.to_numpy()
  df2=np.reshape(df,(28,-1))
  df2.shape
  cv2_imshow(df2)

In [ ]:
# train.head()
y_train=train['45']
# y_train
x_train=train.drop(['45'],axis=1)
y_test=test['45']
x_test=test.drop(['45'],axis=1)
x_train=x_train.to_numpy()
x_test=x_test.to_numpy()

# x_train=x_train[0]
# print(x_train.shape)
# # for img in x_train:
# print(x_train.shape)
# x_train=x_train.reshape(28,28)
# cv2_imshow(x_train)
# x_train=np.rot90(x_train)
# cv2_imshow(x_train)
# print(y_train[0])

In [ ]:
def rotate(image):
    image = image.reshape([28, 28])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image.reshape([28 * 28])
# x_train = np.apply_along_axis(rotate, 1, x_train)
# x_test = np.apply_along_axis(rotate, 1, x_test)
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:
x_train=x_train.astype('float32')
x_train.dtype
x_test=x_test.astype('float32')
x_train/=255
x_test/=255

In [ ]:
x_test.shape

(10223, 28, 28, 1)

In [ ]:
import keras
import tensorflow as tf
import cv2
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
from tensorflow.keras.utils import to_categorical


In [ ]:
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)
train_labels.shape
# print(np.unique(y_test))

(56095, 47)

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(32,(3,3),padding='same'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(64,(3,3),padding="same"))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(256,(3,3),padding="same",activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(47, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.output_shape

(None, 47)

In [ ]:
model.fit(x_train, train_labels, batch_size=5, epochs=25, validation_split=0.1)

Epoch 1/25
10097/10097 [==============================] - 50s 5ms/step - loss: 0.7528 - accuracy: 0.7814 - val_loss: 4.1051 - val_accuracy: 0.1490
Epoch 2/25
10097/10097 [==============================] - 45s 4ms/step - loss: 0.4324 - accuracy: 0.8770 - val_loss: 27.2377 - val_accuracy: 0.1774
Epoch 3/25
10097/10097 [==============================] - 45s 4ms/step - loss: 0.3358 - accuracy: 0.9061 - val_loss: 13.0693 - val_accuracy: 0.1488
Epoch 4/25
10097/10097 [==============================] - 45s 4ms/step - loss: 0.2688 - accuracy: 0.9257 - val_loss: 0.7212 - val_accuracy: 0.7952
Epoch 5/25
10097/10097 [==============================] - 44s 4ms/step - loss: 0.2232 - accuracy: 0.9377 - val_loss: 0.7469 - val_accuracy: 0.7882
Epoch 6/25
10097/10097 [==============================] - 45s 4ms/step - loss: 0.1973 - accuracy: 0.9447 - val_loss: 0.6822 - val_accuracy: 0.8171
Epoch 7/25
10097/10097 [==============================] - 44s 4ms/step - loss: 0.1808 - accuracy: 0.9503 - val_loss:

KeyboardInterrupt: ignored

In [ ]:
score = model.evaluate(x_test, test_labels, verbose=1)

320/320 [==============================] - 1s 4ms/step - loss: 0.1997 - accuracy: 0.9661


In [ ]:
model.save("mcombo.h5")

In [ ]:
model = keras.models.load_model("mcombo.h5")

In [ ]:
def prepro(gray):
    cv2_imshow(gray)
    kernel = np.ones((3,3), np.uint8)
    gray= cv2.erode(gray, kernel, iterations=1)
    # gray= cv2.dilate(gray, kernel, iterations=1)
    # cv2_imshow(gray)
    blur=cv2.GaussianBlur(gray,(5,5),2)
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2_imshow(thresh)
    
    print(thresh.shape)
    h=thresh.shape[0]
    w=thresh.shape[1]
    prev=0
    cont=0
    v=[]
    for i in range(0,w):
        x=0
        for j in range(0,h):
             if(thresh[j][i]==0):
                 x+=1
        if(x==0 and cont==1):
            if(i-prev<10):
               cont=0
            else:
                  cont=0
                  v.append((prev,i))
        elif(x>0 and cont==0):
              prev=i
              cont=1
    
    # for x in v:
    #     cv2.line(img,(x[0],0),(x[0],h-1),(255,0,0),3)
    #     cv2.line(img,(x[1],0),(x[1],h-1),(255,0,0),3)
    
    prev=0        
    u=[]
    for x in v:
          s=x[0]
          e=x[1]
          cont=0
          for i in range(0,h):
               z=0
               for j in range(s,e+1):
                   if(thresh[i][j]==0):
                       z+=1
               if(z>0 and cont==0):
                    prev=i
                    cont=1
               elif(z==0 and cont==1):
                    if(i-prev<10):
                         cont=0
                    else:
                      u.append((prev,i))
                      break
    print(v)
    print(u)
    v2=[]
    u2=[]
    i=0
    while i<len(v)-1:
          if(v[i+1][0]-v[i][1]<3):
               v2.append((v[i][0],v[i+1][1]))
               u2.append((min(u[i][0],u[i+1][0]),max(u[i][1],u[i+1][1])))
               i+=2
          else:
             v2.append(v[i])
             u2.append(u[i])
             i+=1
    if(i==len(v)-1):
        v2.append(v[-1])
        u2.append(u[-1])
    
    st_list=[]
    end_list=[]
    for i in range(len(v2)):
       if((v2[i][1]-v2[i][0])>30 and u2[i][1]-u2[i][0]>30):
           cv2.rectangle(img,(v2[i][0],u2[i][0]),(v2[i][1],u2[i][1]),(255,0,0),3)
           st_list.append((u2[i][0],v2[i][0]))
           end_list.append((u2[i][1],v2[i][1]))
    cv2_imshow(img)    
    return thresh,st_list,end_list
    

In [ ]:

dft=x_test.loc[x_test['45']=='10']
img=x_test[0]
y=model.predict(img.reshape(-1,28,28,1))
prediction = y.argmax(axis=1)
prediction
print(y_test[0])

In [ ]:
classes = "1,2,3,4,5,6,7,A,B,d,e,g,G,Z,W,H,I,J,R,t,K,M,S,N,X,b"
index =   "1,2,3,4,5,6,7,10,11,38,39,41,16,35,32,17,18,19,27,46,20,22,28,23,33,37"
values = classes.split(',')
keys = index.split(',')
map = dict(zip(keys, values))
img = cv2.imread('test9.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2,st,end=prepro(gray)

In [ ]:
fans=''
for i in range(len(st)):
      eximg=img2[st[i][0]:end[i][0],st[i][1]:end[i][1]]
      eximg=cv2.copyMakeBorder(eximg,40,40,40,40,cv2.BORDER_CONSTANT,value=(255,255,255))
      kernel = np.ones((5, 5), np.uint8)
      eximg2=cv2.dilate(eximg,kernel,10)
      cv2_imshow(eximg2) 
      eximg=eximg/255
      eximg=cv2.resize(eximg,(28,28))
      ans=model.predict(eximg.reshape(-1,28,28,1))
      prediction = ans.argmax(axis=1)
      conf=np.max(ans)
      print(prediction,conf)
      fans+=map[str(prediction[0])]

print(fans)